## Copernicus global land cover
Available 2015 to 2019

In [1]:
import ee, geemap

In [2]:
ee.Initialize()

In [3]:
br = (ee.FeatureCollection("FAO/GAUL/2015/level1")
       .filterMetadata('ADM0_NAME', 'equals', 'Brazil')
       .filterMetadata('ADM1_NAME', 'equals', 'Roraima')
      )

In [5]:
Map = geemap.Map()
Map.addLayer(br, {}, 'Amazonas')
Map.center_object(br, zoom=6)
Map

Map(center=[2.082465819696796, -61.39215640405955], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
# Create grid
# https://developers.google.com/earth-engine/tutorials/community/drawing-tools

def make_grid(region, a_scale):
    """
    Creates a grid around a specified ROI.
    User inputs their reasonably small ROI.
    User inputs a scale where 100000 = 100km.
    """
    # Creates image with 2 bands ('longitude', 'latitude') in degrees
    lonLat = ee.Image.pixelLonLat()

    # Select bands, multiply times big number, and truncate
    lonGrid = (lonLat
               .select('latitude')
               .multiply(10000000)
               .toInt()
              )
    latGrid = (lonLat
              .select('longitude')
              .multiply(10000000)
              .toInt()
              )

    # Multiply lat and lon images and reduce to vectors
    grid = (lonGrid
            .multiply(latGrid)
            .reduceToVectors(
                geometry = region,
                scale = a_scale, # 100km-sized boxes needs 100,000
                geometryType = 'polygon')
           )
    
    return(grid)

In [7]:
# Make test grid (half degree squares)
grid1 = make_grid(br, 40000)

In [12]:
Map = geemap.Map()
Map.addLayer(grid1, {}, 'grid')
Map.center_object(br, zoom=5)
Map

Map(center=[2.082465819696796, -61.39215640405955], controls=(WidgetControl(options=['position', 'transparent_…

In [13]:
# Access coordinates of grid squares
grid_dict = grid1.getInfo()

feats = grid_dict['features']
coord_list = []
for d in feats:
    geom = d['geometry']
    coords = geom['coordinates']
    coord_list.append(coords)

In [14]:
# Create a list of several ee.Geometry.Polygons
polys = []
for coord in coord_list:
    poly = ee.Geometry.Polygon(coord)
    polys.append(poly)

In [15]:
# Make grid smaller if it's huge
idx = [10]
polys = [polys[i] for i in idx]

In [16]:
# Make the whole grid a feature collection for export purposes
grid = ee.FeatureCollection(polys[0])

In [17]:
Map = geemap.Map()
Map.addLayer(grid, {}, 'grid')
Map.center_object(grid, zoom=10)
Map

Map(center=[0.8983167552200919, -59.82779792235959], controls=(WidgetControl(options=['position', 'transparent…

In [18]:
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
             .select('ECO_NAME', 'BIOME_NUM', 'COLOR')
             .filterBounds(grid))

def setcolor(f):
    color = f.get('COLOR')
    return f.set('style', {'color': color, 'width': 0})

ecoregions = ecoregions.map(setcolor)
imageRGB = ecoregions.style(styleProperty='style')
Map.addLayer(imageRGB, {}, 'fills')

In [19]:
lc = (ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019")
      .select('discrete_classification')
     .clip(polys[0]))
Map.addLayer(lc, {}, 'lc')

In [20]:
empty = ee.Image().byte()
outline = empty.paint(
    featureCollection=grid,
    color=1,
    width=3)
Map.addLayer(outline, {}, 'outline')